In [2]:
import numpy as np
import xarray as xr

# Load environmental dust cube
cube = np.load("../outputs/environmental_dust_cube.npy")

# Create xarray dataset
dust_ds = xr.Dataset(
    {
        "NDVI": (("time", "y", "x"), cube[..., 0]),
        "Wind Speed": (("time", "y", "x"), cube[..., 1]),
        "Wind Direction": (("time", "y", "x"), cube[..., 2]),
        "Dust Activity Index": (("time", "y", "x"), cube[..., 3]),
    },
    coords={
        "time": np.arange(cube.shape[0]),
        "y": np.arange(cube.shape[1]),
        "x": np.arange(cube.shape[2]),
    }
)

print("dust_ds loaded successfully")
print(dust_ds)


dust_ds loaded successfully
<xarray.Dataset> Size: 233MB
Dimensions:              (time: 14, y: 721, x: 1440)
Coordinates:
  * time                 (time) int32 56B 0 1 2 3 4 5 6 7 8 9 10 11 12 13
  * y                    (y) int32 3kB 0 1 2 3 4 5 6 ... 715 716 717 718 719 720
  * x                    (x) int32 6kB 0 1 2 3 4 5 ... 1435 1436 1437 1438 1439
Data variables:
    NDVI                 (time, y, x) float32 58MB 0.5 0.5 0.5 ... 0.5 0.5 0.5
    Wind Speed           (time, y, x) float32 58MB 6.318 6.318 ... 4.691 4.691
    Wind Direction       (time, y, x) float32 58MB 1.222 1.222 ... -0.3949
    Dust Activity Index  (time, y, x) float32 58MB 3.159 3.159 ... 2.345 2.345


In [8]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import webbrowser
import threading

# =========================
# CREATE DASH APP
# =========================
app = Dash(__name__)

# =========================
# DASHBOARD LAYOUT
# =========================
app.layout = html.Div(
    style={"backgroundColor": "#111111", "minHeight": "100vh", "padding": "15px"},
    children=[

        # -------- HEADER --------
        html.H1(
            "Interactive Dust Forecasting & Analysis System",
            style={"textAlign": "center", "color": "white"}
        ),

        html.P(
            "Explore dust sources, transport dynamics, and activity levels "
            "using spatio-temporal environmental data.",
            style={"textAlign": "center", "color": "#CCCCCC"}
        ),

        # -------- MAIN SECTION --------
        html.Div(
            style={"display": "flex", "gap": "15px"},
            children=[

                # -------- LEFT CONTROL PANEL --------
                html.Div(
                    style={
                        "width": "25%",
                        "padding": "15px",
                        "backgroundColor": "#1e1e1e",
                        "borderRadius": "10px",
                    },
                    children=[
                        html.Label("Select Variable", style={"color": "white"}),

                        dcc.Dropdown(
                            id="variable-dropdown",
                            options=[{"label": v, "value": v} for v in dust_ds.data_vars],
                            value=list(dust_ds.data_vars)[0],
                            clearable=False,
                        ),

                        html.Br(),

                        html.Label("Select Time Step", style={"color": "white"}),

                        dcc.Slider(
                            id="time-slider",
                            min=0,
                            max=len(dust_ds.time) - 1,
                            step=1,
                            value=0,
                            tooltip={"placement": "bottom", "always_visible": True},
                        ),

                        html.Br(),
                        html.Div(
                            "Tip: Click on the map to view local trends",
                            style={"color": "#AAAAAA", "fontSize": "12px"}
                        ),
                    ],
                ),

                # -------- SPATIAL MAP --------
                html.Div(
                    style={"width": "75%", "padding": "10px"},
                    children=[
                        dcc.Graph(
                            id="spatial-map",
                            style={"height": "75vh"},
                            config={"displayModeBar": False},
                        )
                    ],
                ),
            ],
        ),

        html.Hr(style={"borderColor": "#333"}),

        # -------- TIME SERIES --------
        html.Div(
            children=[
                dcc.Graph(id="time-series")
            ]
        )
    ],
)

# =========================
# SPATIAL MAP CALLBACK
# =========================
@app.callback(
    Output("spatial-map", "figure"),
    Input("variable-dropdown", "value"),
    Input("time-slider", "value")
)
def update_spatial_map(variable, t):

    data = dust_ds[variable].isel(time=t).values

    fig = px.imshow(
        data,
        origin="upper",
        aspect="equal",
        color_continuous_scale="Turbo",
    )

    fig.update_layout(
        title=f"{variable} — Time Step {t}",
        template="plotly_dark",
        paper_bgcolor="#111111",
        plot_bgcolor="#111111",
        margin=dict(l=20, r=20, t=50, b=20),
    )

    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)

    return fig


# =========================
# TIME SERIES CALLBACK
# =========================
@app.callback(
    Output("time-series", "figure"),
    Input("spatial-map", "clickData"),
    Input("variable-dropdown", "value")
)
def update_time_series(clickData, variable):

    if clickData is None:
        y = dust_ds.dims["y"] // 2
        x = dust_ds.dims["x"] // 2
    else:
        x = int(clickData["points"][0]["x"])
        y = int(clickData["points"][0]["y"])

    ts = dust_ds[variable][:, y, x].values

    fig = go.Figure()
    fig.add_trace(go.Scatter(y=ts, mode="lines", line=dict(color="orange")))

    fig.update_layout(
        title=f"{variable} Time Series (y={y}, x={x})",
        template="plotly_dark",
        paper_bgcolor="#111111",
        plot_bgcolor="#111111",
        height=350,
    )

    return fig


# =========================
# AUTO-OPEN BROWSER & RUN
# =========================
def open_browser():
    webbrowser.open_new("http://127.0.0.1:8051/")

threading.Timer(1, open_browser).start()
app.run(debug=False, port=8051)


C:\Users\Haneesh.B\AppData\Local\Temp\ipykernel_33064\1395582274.py:144: FutureWarning:

The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.

C:\Users\Haneesh.B\AppData\Local\Temp\ipykernel_33064\1395582274.py:145: FutureWarning:

The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.

C:\Users\Haneesh.B\AppData\Local\Temp\ipykernel_33064\1395582274.py:144: FutureWarning:

The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.

C:\Users\Haneesh.B\AppData\Local\Temp

In [7]:
import webbrowser
import threading

def open_browser():
    webbrowser.open_new("http://127.0.0.1:8051/")

# Open browser 1 second after server starts
threading.Timer(1, open_browser).start()

app.run(debug=False, port=8051)
